<a href="https://colab.research.google.com/github/beatrizugalde/Tesina/blob/main/Copia_de_Visualizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar librerías:

import pandas as pd
#import json as jd
#import plotly as plt

1. Extraer características desde el archivo original en xml. Se elimina la altitud de las coordenadas. Se guarda en excel y csv

In [ ]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd


kml_files = ['/content/usolospinos.kml', '/content/usolaspalmas.kml']  # Cambiar a carpeta en github o a rutas en github de los predios

terrenos = []

# Procesar un archivo KML
def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Espacio de nombres KML
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Iterar sobre cada <Placemark>
    placemarks = root.findall('.//kml:Placemark', ns)
    for placemark in placemarks:
        # Extraer el nombre y descripción
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text

        # Procesar la descripción para extraer la segunda tabla
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        second_table = tables[1] if len(tables) > 1 else None

        # Almacenar los pares clave-valor de la segunda tabla
        info_terreno = {'nombre': nombre}
        if second_table:
            rows = second_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    info_terreno[key] = value

        # Extraer las coordenadas
        coordenadas_texto = placemark.find('.//kml:coordinates', ns).text.strip()
        coordenadas = [(float(coord.split(',')[0]), float(coord.split(',')[1]))#dejar fuera la altitud
        for coord in coordenadas_texto.split()]
        info_terreno['coordenadas'] = coordenadas

        terrenos.append(info_terreno)

# Procesar todos los archivos KML
for kml_file in kml_files:
    procesar_kml(kml_file)


df= pd.DataFrame(terrenos)

df["ID"] = df["ID_PREDIO"].astype(str) + df["ID_UNICO"].astype(str)

df.to_excel('terrenos.xlsx', index=False)

df.to_csv('terrenos.csv', index=False)

print("Datos exportados exitosamente a Excel y CSV.")

Datos exportados exitosamente a Excel y CSV.


2. Análisis del df "predios"

In [ ]:
df['NOMBRE'].unique()

array(['Los Pinos', 'Las Palmas'], dtype=object)

In [ ]:
valores_unicos = df["ID"].unique()
len(valores_unicos)

580

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nombre       580 non-null    object
 1   FID          580 non-null    object
 2   ID_UNICO     580 non-null    object
 3   ID_PREDIO    580 non-null    object
 4   NOMBRE       580 non-null    object
 5   ID_USO       580 non-null    object
 6   ID_SUBUSO    580 non-null    object
 7   ID_ESTRUC    580 non-null    object
 8   ID_TIFO      580 non-null    object
 9   ID_STIF      580 non-null    object
 10  ID_ESP1      580 non-null    object
 11  ID_ESP2      580 non-null    object
 12  ID_ESP3      580 non-null    object
 13  ID_RODAL     580 non-null    object
 14  APLANT       580 non-null    object
 15  ACOSECHA     580 non-null    object
 16  AREVISION    580 non-null    object
 17  ID_ADM       580 non-null    object
 18  ID_USOPRE    580 non-null    object
 19  ID_MANEJO    580 non-null    

In [ ]:
df.head(1)

,nombre,FID,ID_UNICO,ID_PREDIO,NOMBRE,ID_USO,ID_SUBUSO,ID_ESTRUC,ID_TIFO,ID_STIF,...,ID_RALEO,ID_PODA,ID_SANID,ID_ROTAC,USO_COT,USO_CEFOR,OBS,SUP_HA,coordenadas,ID
0,01,0,1,01,Los Pinos,04,01,01,,,...,,,,,Plantacion Adulta,Plantacion,,0.41,"[(-73.16706602941588, -39.73334469547343), (-7...",011


In [ ]:
#Crear el geojson. el feature id va a ser la columna ID del df
import json
#import plotly.express as px

geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for index, row in df.iterrows():
    feature = {
        "type": "Feature",
        "properties": {
            "id": row["ID"],
            "ID_ESP1": row.get("ID_ESP1", "N/A")  # Asegúrate de tener esta columna en el DataFrame
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [row["coordenadas"]]
        },
        "id":row["ID"]
    }
    geojson_data["features"].append(feature)

with open("terrenos.geojson", "w") as geojson_file:
    json.dump(geojson_data, geojson_file)


In [ ]:
import plotly.express as px

In [ ]:
#from urllib.request import urlopen
#import json
#with urlopen('https://raw.githubusercontent.com/beatrizugalde/Tesina/refs/heads/main/terrenos%20(4).geojson') as response:
#    counties = json.load(response)

#import pandas as pd
#df = pd.read_csv("https://raw.githubusercontent.com/beatrizugalde/Tesina/refs/heads/main/terrenos%20(2).csv",dtype={"ID": str})

#import plotly.express as px

#fig = px.choropleth_map(df, geojson=counties, locations='ID',color='ID_ESP1',featureidkey='id',center={'lat':-39.73,"lon":-73.17},map_style="carto-positron",zoom=10,opacity=0.5)
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

In [ ]:
#Modificar el df: quedarme con las columnas importantes solamente

quedar =['ID','NOMBRE','ID_UNICO','SUP_HA','ID_ESP1','USO_COT','USO_CEFOR']

In [ ]:

df = df[quedar]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         580 non-null    object 
 1   NOMBRE     580 non-null    object 
 2   ID_UNICO   580 non-null    int64  
 3   SUP_HA     580 non-null    float64
 4   ID_ESP1    557 non-null    object 
 5   USO_COT    580 non-null    object 
 6   USO_CEFOR  580 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 31.8+ KB


In [ ]:
# convierte la columna ID_UNICO a tipo str
df['ID_UNICO'] = df['ID_UNICO'].astype(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         580 non-null    object 
 1   NOMBRE     580 non-null    object 
 2   ID_UNICO   580 non-null    object 
 3   SUP_HA     580 non-null    float64
 4   ID_ESP1    557 non-null    object 
 5   USO_COT    580 non-null    object 
 6   USO_CEFOR  580 non-null    object 
dtypes: float64(1), object(6)
memory usage: 31.8+ KB


In [ ]:
df["NOMBRE"].unique()

array(['Los Pinos', 'Las Palmas'], dtype=object)

In [ ]:
df["ID_ESP1"].unique()

array(['ND', 'AG', 'NO', 'CS', 'ME', 'Cq', 'PR', 'NI', nan, 'EG', 'Ue',
       'hl', 'EC', 'AD', 'PO', 'ju', 'SA', 'AR', 'Mq', 'CU', 'GA', 'SQ',
       'AP', 'CE', 'DW', 'Ru', 'PA', 'AB', 'PM', 'PT', 'PY', 'AM'],
      dtype=object)

In [ ]:
df["USO_COT"].unique()

array(['Plantacion Adulta', 'Bosque Renoval',
       'Bosque Exoticas Asilvestradas',
       'Bosque Nativo con Exoticas Asilvestradas',
       'Matorral Arborescente', 'Matorral', 'Bosque Adulto Renoval',
       'Plantacion Joven-Recien Cosechada', 'Matorral-Pradera',
       'Bosque Nativo Plantacion', 'Otros Terrenos Humedos',
       'Pradera Perenne', 'Areas sin Vegetacion', 'Vegas', 'Lago Laguna'],
      dtype=object)

In [ ]:
df["USO_CEFOR"].unique()

array(['Plantacion', 'Bosque Nativo', 'Bosque Exoticas Asilvestradas',
       'Bosques Mixtos', 'Matorral', 'Reforestacion', 'Terrenos Humedos',
       'Pradera', 'Faja Camino', 'Cuerpos de Agua'], dtype=object)

In [ ]:
especies = df["ID_ESP1"].unique()
len(especies)

32

In [ ]:
id_counts_per_species = df.groupby('ID_ESP1')['ID'].count()
id_counts_per_species

In [ ]:
A=0
for e in id_counts_per_species:
  A=A+e

print(A)

557


In [ ]:
empty_id_esp1_count = df['ID_ESP1'].isnull().sum()
print(f"Number of empty 'ID_ESP1' values: {empty_id_esp1_count}")

Number of empty 'ID_ESP1' values: 23


In [ ]:
print(A+empty_id_esp1_count)

580


In [ ]:
#después hacerlo sort
fig = px.bar(id_counts_per_species,
             x=id_counts_per_species.index,
             y=id_counts_per_species.values,
             labels={'x': 'Species', 'y': 'Count of IDs'},
             title='Count of IDs per Species',text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
suma_ha_especies = df.groupby('ID_ESP1')['SUP_HA'].sum()
suma_ha_especies

In [ ]:
fig = px.bar(suma_ha_especies,
             x=suma_ha_especies.index,
             y=suma_ha_especies.values,
             labels={'x': 'Species', 'y': 'Hectáreas'},
             title='Hectáreas por especie', text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
fig = px.choropleth_map(df, geojson=counties, locations='ID',color='USO_CEFOR',featureidkey='id',center={'lat':-39.73,"lon":-73.17},map_style="carto-positron",zoom=10,opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Buffered data was truncated after reaching the output size limit.

In [ ]:
recuento_uso = df.groupby('USO_CEFOR')['ID'].count()

fig = px.bar(recuento_uso,
             x=recuento_uso.index,
             y=recuento_uso.values,text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
suma_uso = df.groupby('USO_CEFOR')['SUP_HA'].sum()

fig = px.bar(suma_uso,
             x=suma_uso.index,
             y=suma_uso.values,text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
# prompt: genera un unevo df agrupando las categorías: USO_CEFOR y ID_ESP1, el valor final es la suma de SUP_HA de cada grupo

# Agrupar por 'USO_CEFOR' e 'ID_ESP1' y sumar 'SUP_HA'
unevo_df = df.groupby(['USO_CEFOR', 'ID_ESP1'])['SUP_HA'].sum().reset_index()

unevo_df

In [ ]:
unevo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   USO_CEFOR  45 non-null     object 
 1   ID_ESP1    45 non-null     object 
 2   SUP_HA     45 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.2+ KB


In [ ]:
fig = px.bar(unevo_df,
             x="USO_CEFOR",
             y="SUP_HA",color="ID_ESP1",text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()